In [ ]:
from nbtemplate import display_header, display_codetoggle, get_path
display_header('ALookAtRalfsGratings.ipynb')

## Grating efficiencies

This short write-up collects a few different ways to visualize the grating efficiencies that Ralf
has provided for my simulations.
Looking at those plots helps to understand which area of the Rowland circle shold be covered with detectors
and which of them receive hardly any photons anyway.

In [ ]:
import numpy as np
import astropy.units as u
from cycler import cycler
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
plt.rc('axes', prop_cycle=(cycler('color', ['r', 'g', 'b', 'y', 'c', 'm', 'k', 'orange', 'lime']) *
cycler('linestyle', ['-', '--', ':'])))


In [ ]:
import arcus

In [ ]:
# Make wavelength grid for plots
wave = np.arange(8., 50., 0.5) * u.Angstrom
energies = wave.to(u.keV, equivalencies=u.spectral()).value

In [ ]:
d = 2e-4 * u.mm

ralftab = arcus.ralfgrating.InterpolateRalfTable()

def plot_wave_orders(ax, blazeang, x='wave', plot_orders=range(-12, 1)):
    orders, inter = ralftab.probabilities(energies, None, np.deg2rad(blazeang))
    for o in plot_orders:
        n = (orders == o)
        if n.sum() != 1:
            raise ValueError(f'order {o} not defined in the probabilities table.')
        else:
            n = n.nonzero()[0][0]
        if x == 'wave':
            ax.plot(wave, inter[n, :], label=o)
        elif x == 'angle':
            ax.plot(np.arcsin(n * wave / d), inter[n, :], label=o)
        elif x == 'mlambda':
            ax.plot(wave * np.abs(orders[n]), inter[n, :], label=o)
    if x == 'wave':
        ax.set_xlabel('wavelength [$\AA{}$]')
    elif x == 'angle':
        ax.set_xlabel('grating angle [rad]')
    elif x == 'mlambda':
        ax.set_xlabel(' m * $\lambda$ [$\AA{}$]')
    ax.set_ylabel('grating efficiency')
    ax.legend()

In [ ]:
fig = plt.figure(figsize=(12, 12))
for i, blazeang in enumerate([1.91, 1.7, 1.5, 1.3]):
    ax = fig.add_subplot(2,2,i+1)
    plot_wave_orders(ax, blazeang, 'wave')
    ax.set_xlim([None, 60])
    ax.set_title('blaze angle = {0}'.format(blazeang))


### Second peak far below blaze peak
**Question**: Order 4 has peak at  at 33 Ang. Why is there a second peak about 12 Ang?
Order 3 has blaze peak that 45 Ang. Why is there a second peak at 12 Ang?
Order 2 has peak apparently > 50 Ang (off the plot). Why is there a second large peak around 12 Ang?

**Answer from Ralf**
At these short wavelengths the attenuation length in Si becomes comparable to or greater than the depth of the grating (4 micron), i.e. the grating becomes partially transparent, or in other words, partially transmits through the silicon bars.  This turns the grating into more of a phase-shifting grating, similar to the HETG with Au bars at higher energies.  Or like a glass grating for visible light.  It's a gradual change as you go towards shorter wavelengths, and so there are regions where 3rd and 4th order are strong, just as 1st order peaks at a certain energy for HETG.

Hope this helps.

Ralf


In [ ]:
fig = plt.figure(figsize=(12, 12))
for i, blazeang in enumerate([1.91, 1.7, 1.5, 1.3]):
    ax = fig.add_subplot(2,2,i+1)
    plot_wave_orders(ax, blazeang, 'angle')
    ax.set_title('blaze angle = {0}'.format(blazeang))
    ax.set_xlim([0, 0.12])

The blaze peak is around 0.07 rad. Obviously this is the region that should be covered with CCDs to obtain
the best possible effective area.

However, for the low orders, most of the effictive area is actually not at the blaze peak, but at much smaller angles close to the zeroth order. If we want to go to low wavelength (high energies) then we have to record that signal, too.

In [ ]:
np.rad2deg(0.07)

In [ ]:
fig = plt.figure(figsize=(12, 12))
for i, blazeang in enumerate([1.91, 1.7, 1.5, 1.3]):
    ax = fig.add_subplot(2,2,i+1)
    plot_wave_orders(ax, blazeang, 'mlambda')
    ax.set_title('blaze angle = {0}'.format(blazeang))
    ax.set_xlim([0, 250])

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plot_wave_orders(ax, 1.91, 'mlambda')
ax.set_title('blaze angle = {0}'.format(1.91))
ax.set_xlim([0, 250])

In [ ]:
orders, inter = ralftab.probabilities(energies, None, np.deg2rad(1.9))

In [ ]:
for blaze in [1.3, 1.5, 1.7, 1.9]:
    orders, inter = ralftab.probabilities(energies, None, np.deg2rad(blaze))
    plt.plot(wave, inter[orders < 0, :].sum(axis=0), label='{:3.1f} deg'.format(blaze))
    
out = plt.legend(title='blaze')
out = plt.title('Sum of dispersed orders')

This plot (above) shows the summed efficiency of all dispersed (negative) orders. We'll look at the positive orders below, but for CAT gratings there is very little power in the positive orders anyway, because they are blazed. Different lines in the plot compare different blaze angles. For the range of angles between about 1.3 and 1.9 degrees, the efficiencies are typically within about 10 % of each other, but the best overall efficiency is achieved for a blaze angle around 1.6 deg. Note that this is just the grating efficiency. For instrument design, we have to take into account that not all orders might be covered by CCDs.

In [ ]:
fig = plt.figure()

ax = fig.add_subplot(111)
plot_wave_orders(ax, 1.91, 'wave', plot_orders=np.arange(-4, 5))
ax.set_xlim([None, 60])
out = ax.set_title('Looking and anti-blaze orders')

In this last plot, we chose the plot range, to highlight orders in the anti-blaze direction. There is not much power there for CAT gratings, but it's non-zero at least for order +1 for a wide range of energies and there is some contribution for +2. Higher orders are essentially negligible. 

In [ ]:
display_codetoggle()